## Importing modules and packages

In [1]:
# Importing packages
import osmnx  as ox
import pandas as pd
import numpy  as np
import folium
import os.path
import shapely
import geopandas as gpd
from IPython.display import IFrame

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching of roads
import gr_placematch # Contains functions that perform the map matching of places
import gr_utils # Contains useful geometry functions
import gr_plot # Contains plotting routines
import gr_process
import roadmatch

# Configuring modules & packages
ox.settings.useful_tags_way = [
    "bridge","tunnel","name","highway","area","landuse","surface","tracktype"
] # Configuring which parameters we want to obtain from OSM

## Input parameters

In [2]:
trailname = 'gr131' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)

# Settings for trail2roads
points_per_batch = 100 # Subdivide the trail into batches of this many points
delta_roads = 0.010 # Tolerance around bounding box per trail section [deg]
min_dist_from_bbox = 0.005 # [deg]
npaths = 5 # number of paths to generate per piece when pathfindinng

# Settings for roads2places
points_per_batch_places = 100 # Subdivide the trail into batches of this many segments
delta_places = 0.015 # bbox delta in deg
buffersize = 0.00015 # buffer used when merging landuse places
tol_area = 15.0e-6 # buffer used to define which merged landuse places we consider

# Settings for places2processed
tol_d        = 0.75 # Consider a segment developed if it lies closer than tol_d to a developed area
tracktype_p0 = ['grade4','grade5']
tracktype_p1 = ['grade2','grade3']
tracktype_p2 = ['grade1']
surface_p0   = ['ground','grass','dirt','sand','earth','mud']
surface_p1   = ['unpaved','gravel','fine_gravel','wood','compacted','rocks','pebblestone','woodchips','snow','ice','salt']
highway_p1   = ['track','path','footway','bridleway']
types_slow   = ['pedestrian','track','footway','bridleway','steps','corridor','path']
types_heavy  = ['motorway','trunk','primary','secondary','tertiary']

## Loading GPX file

In [3]:
gpx = gr_utils.get_gpx(trailname)
gpx_coords = gr_mapmatch.trail_to_coords(gpx) # Convert the points into a list of [lat, lon] pairs

Loading trail points from <data_input/gr131.gpx>...
Finished loading.


## Matching GPX points to OSM nodes

In [5]:
# Generate the nodes dataframe [point_x, point_y, node_id, node_x, node_y, d2node]
# with point = a GPX point and node = the corresponding node
# and d2node = the distance from the GPX point to its corresponding node
filename_nodes = 'cache/' + trailname + '_nodes.csv'
if not os.path.isfile(filename_nodes): # The nodes file does not exist, construct it
    nodes = roadmatch.gpx_to_nodes(gpx, gpx_coords, points_per_batch, delta_roads, min_dist_from_bbox)
    nodes.to_csv(filename_nodes)
else: # The nodes file exists, load it
    nodes = pd.read_csv(filename_nodes,index_col=0,
                            dtype={'point_x':float, 'point_y':float,
                                   'node_id':int,
                                   'node_x':float, 'node_y':float,
                                   'd2node':float})
    
# Generate the pieces dataframe [node0, node1, gpx0, gpx1]
# it cuts up the GPX trail into pieces on which individual pathfinding can be done
filename_pieces = 'cache/' + trailname + '_pieces.csv'
if not os.path.isfile(filename_pieces): # The pieces file does not exist, construct it
    pieces = roadmatch.nodes_to_pieces(nodes)
    pieces.to_csv(filename_pieces)
else:
    pieces = pd.read_csv(filename_pieces,index_col=0,
                         dtype={'node0':int,'node1':int,
                                'gpx0':int,'gpx1':int})
    
# Generate the segments dataframe [x0,y0,x1,y1,d_cart,d_osm,highway,surface,tracktype]
filename_segments = 'cache/' + trailname + '_segments.csv'
if not os.path.isfile(filename_segments): # The pieces file does not exist, construct it
    segments = roadmatch.pieces_to_segments(gpx,nodes,points_per_batch,delta_roads,pieces,npaths)
    segments.to_csv(filename_segments)
else:
    segments = pd.read_csv(filename_segments,index_col=0,
                         dtype={'x0':float,'y0':float,
                                'x1':float,'y1':float,
                               'd_cart':float,'d_osm':float,
                                'highway':str,'surface':str,'tracktype':str})

## Gathering place information

In [6]:
filename_places = 'cache/' + trailname + '_places.csv'

if not os.path.isfile(filename_places): # The merged PLACES file does not exist
    
    print('Merged PLACES file was not found, merging and saving...')
    gr_placematch.roads2places(trailname,segments,points_per_batch_places, delta_places, buffersize, tol_area)
    data_places = gr_utils.merge_places(trailname, segments, points_per_batch_places) # Merge the different sections
    gr_utils.write_places(trailname, data_places) # Write the merged PLACES data
    print('Saved.')
    
else: # The merged PLACES file does exist
    
    print('Loading merged PLACES file...')
    data_places = gr_utils.read_places(trailname) # Read the merged PLACES file
    print('Loaded.')

Merged PLACES file was not found, merging and saving...
Handling batch 0 of 41 that covers road segments 0 through 99...


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


   Writing outputs to file...
   Finished this batch.
Handling batch 1 of 41 that covers road segments 100 through 199...


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


   Writing outputs to file...
   Finished this batch.
Handling batch 2 of 41 that covers road segments 200 through 299...


...

   Writing outputs to file...
   Finished this batch.
Handling batch 40 of 41 that covers road segments 4000 through 4099...


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


   Writing outputs to file...
   Finished this batch.
Handling batch 41 of 41 that covers road segments 4100 through 4170...


/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))
/Users/svanwal/Documents/Programming/Projects/gr-analysis/gr_placematch.py:107: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  merged_polys = list(shapely.ops.unary_union(polys))


   Writing outputs to file...
   Finished this batch.
Saved.


## Evaluating processed properties

In [7]:
filename_processed = 'data_output/' + trailname + '_processed.csv'

if not os.path.isfile(filename_processed): # The PROCESSED file does not exist
    
    print('The PROCESSED file was not found, processing and saving...')
    data_places2 = gr_mapmatch.remove_repeat_segments(data_places) # Remove backtracked sections
    data = gr_process.places2processed(data_places2,tol_d,types_slow,types_heavy,tracktype_p0,tracktype_p1,tracktype_p2,surface_p0,surface_p1,highway_p1) # Determine traffic/development/paved status & GR types
    gr_utils.write_processed(trailname, data) # Write the processed data
    print('Saved.')
    
else: # The PROCESSED file does exist
    
    print('The PROCESSED file already exists, so nothing needs to be done.')

The PROCESSED file was not found, processing and saving...
Saved.
